In [1]:
!pip install -q praat-parselmouth
import glob
import os.path
import parselmouth
import pickle
import re
import itertools
import numpy as np 
import pandas as pd
import csv
from os import listdir
from xlwt import Workbook
from parselmouth.praat import call
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from functools import reduce

     |████████████████████████████████| 9.0MB 2.8MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [43]:
def measureDuration(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    duration = parselmouth.praat.call(pitch, "Get end time")
    return duration

In [50]:
def measurePitch(voiceID, f0min, f0max, unit, startTime, endTime):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    pulses = parselmouth.praat.call([sound, pitch], "To PointProcess (cc)")
    duration = parselmouth.praat.call(pitch, "Get end time")
    voice_report_str = parselmouth.praat.call([sound, pitch, pulses], "Voice report", startTime, endTime, 75, 600, 1.3, 1.6, 0.03, 0.45)
    voice_report_str = voice_report_str.replace("--undefined--", "0")
    s=re.findall(r'-?\d+\.?\d*',voice_report_str)
    if (len(s) == 40):
      report = [s[21], s[22]+'E'+s[23],s[24],s[26],s[27],
              s[28],s[29],s[31],s[33],
              s[35],s[36],s[37],
              s[38],s[39],
              s[3],s[4],s[5],s[6],s[7],s[8],
              s[9],s[10]+'E'+s[11],s[12]+'E'+s[13]]
      return report
    else: 
      report = [s[19], s[20], s[21], s[23], s[24], s[25], 
                s[26], s[28], s[30], s[32], s[33], s[34],
                s[35], s[36], s[3], s[6], s[7], s[8],
                s[9], s[10], s[11]]
      print(voice_report_str)
      print(report)
      return report

In [51]:
def seq(start, end, step):
    if step == 0:
        raise ValueError("step must not be 0")
    sample_count = int(abs(end - start) / step)
    return itertools.islice(itertools.count(start, step), sample_count)

In [ ]:
for wave_file in sorted(glob.glob("/content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/*.wav"), key=lambda x:x[4:]):
  print("Processing {}...".format(wave_file))
  sound = parselmouth.Sound(wave_file)

  stepSize = 5.0
  start_stepSize = 2.5
  praat_features = []

  start_time = 0
  end_time = duration
  sample_count = int(abs(end_time - start_time) / start_stepSize)
  
  for start_time in itertools.islice(itertools.count(start_time, start_stepSize), sample_count):
      if start_time == 0:
         end_time = start_time + stepSize
         audio_Features = []
         praat_features = []
         praat_features.append(measurePitch(sound, 75, 500, "Hertz", start_time, end_time))
         features = [element for tupl in praat_features for element in tupl]
         features = list(map(float, features))
         audioID = str(os.path.basename(wave_file))[0:4]
         audio_Features.append(audioID)
         diagnosis = str(os.path.basename(wave_file))[5:7]
         audio_Features.append(diagnosis.upper())
         duration = measureDuration(sound, 75, 500, "Hertz")
         audio_Features.append(duration)

         completeFeatures = audio_Features + features
         
         with open('audioFeatures.csv', 'a') as audio_Features:
           audio_writer = csv.writer(audio_Features, delimiter=',')
           audio_writer.writerow(completeFeatures)

      elif start_time + (2*start_stepSize) >= duration:
        end_time = measureDuration(sound, 75, 500, "Hertz")
        audio_Features = []
        praat_features = []
        praat_features.append(measurePitch(sound, 75, 500, "Hertz", start_time, end_time))
        features = [element for tupl in praat_features for element in tupl]
        features = list(map(float, features))
        audioID = str(os.path.basename(wave_file))[0:4]
        audio_Features.append(audioID)
        diagnosis = str(os.path.basename(wave_file))[5:7]
        audio_Features.append(diagnosis.upper())
        duration = measureDuration(sound, 75, 500, "Hertz")
        audio_Features.append(duration)

        completeFeatures = audio_Features + features
        
        with open('audioFeatures.csv', 'a') as audio_Features:
          audio_writer = csv.writer(audio_Features, delimiter=',')
          audio_writer.writerow(completeFeatures)

      else:
        end_time = start_time + stepSize
        audio_Features = []
        praat_features = []
        praat_features.append(measurePitch(sound, 75, 500, "Hertz", start_time, end_time))
        features = [element for tupl in praat_features for element in tupl]
        features = list(map(float, features))
        audioID = str(os.path.basename(wave_file))[0:4]
        audio_Features.append(audioID)
        diagnosis = str(os.path.basename(wave_file))[5:7]
        audio_Features.append(diagnosis.upper())
        duration = measureDuration(sound, 75, 500, "Hertz")
        audio_Features.append(duration)

        completeFeatures = audio_Features + features
        
        with open('audioFeatures.csv', 'a') as audio_Features:
          audio_writer = csv.writer(audio_Features, delimiter=',')
          audio_writer.writerow(completeFeatures)


Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID00_hc_0_0_0.wav...
Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID01_hc_0_0_0.wav...
Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID03_hc_0_0_0.wav...
Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID05_hc_0_0_0.wav...
Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID08_hc_0_0_0.wav...
Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID09_hc_0_0_0.wav...
Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID10_hc_0_0_0.wav...
Processing /content/drive/My Drive/Masters_Project/Raw_Wav_Files/Read_Text/Healthy_Control/ID11_hc_0_0_0.wav...
